# Геолокация новой скважины

<div style="border:solid black 2px; padding: 20px">
    
### Описание проекта

Добывающей компании необходимо решить, где бурить скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. **Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.** Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


### Описание данных:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

### Условия задачи:
1. Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
2. При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
3. Бюджет на разработку скважин в регионе — 10 млрд рублей.
4. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
5. После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

## Этапы исследования

[Шаг 1. Загрузка и подготовка данных](#1)

[Шаг 2. Обучение и проверка модели](#2)

[Шаг 3. Подготовка к расчету прибыли](#3)

[Шаг 4. Функция для расчета прибыли по выбранным скважинам и предсказаниям модели](#4)

[Шаг 5. Расчёт прибыли и рисков](#5)

[Шаг 6. Общий вывод](#6)

<a id='1'></a>

# 1. Загрузка и подготовка данных

Откроем и подготовим файлы последовательно.<br>
Сразу проверим на корреляцию.

### 1.1 Первый регион

In [1]:
import pandas as pd

data_1 = pd.read_csv('/datasets/geo_data_0.csv')
display(data_1.head(15))
display(data_1.info())
display(data_1.describe())
data_1.corr()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Можем заметить, что есть скважина, объем запасов в которой равен 0.

Сильной зависимости параметров друг от друга не выявлено.

### 1.2 Второй регион

In [2]:
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
display(data_2.head(15))
display(data_2.info())
display(data_2.describe())
data_2.corr()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


В целом, во втором регионе аналогичная ситуация с пустыми скважинами, однако тут есть сильная зависимость f2-признака от "product", равная почти единице.<br>
Т.е. у нас есть коэффициент их зависимости, равный приблизительно 27.<br>
Высока вероятность того, что в скважине мы найдем объем запасов = 27*f2

### 1.3 Третий регион

In [3]:
data_3 = pd.read_csv('/datasets/geo_data_2.csv')
display(data_3.head(15))
display(data_3.info())
display(data_3.describe())
data_3.corr()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Ситуация аналогичная с 1ым регионом.

**Удалим id всех скважин - для обучения модели это лишний шум**

In [4]:
data_1 = data_1.drop('id', axis =1)
data_2 = data_2.drop('id', axis =1)
data_3 = data_3.drop('id', axis =1)

<a id='2'></a>

# 2. Обучение и проверка модели

### 2.1 Первый регион

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Напишем функцию для обучения модели под каждый регион
def region(data):
    features = data.drop('product', axis = 1)
    target = data['product']
    

    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              test_size = 0.25, 
                                                                              random_state = 12345)
    # Стандартизируем данные
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

    #Обучим модель линейний регрессии
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    
    return predicted_valid, target_valid, rmse

predicted_valid, target_valid, rmse = region(data_1)
print('Средний запас предсказанного сырья:', predicted_valid.mean())
print('Средний запас реального сырья:', target_valid.mean())
print('RMSE:', rmse)

Средний запас предсказанного сырья: 92.59256778438038
Средний запас реального сырья: 92.07859674082927
RMSE: 37.5794217150813


Делаем вывод, что по средним запасам у нас почти что сходство, однако в некоторых скважинах у нас присутствует сильная разница между предсказаниями модели и действительными данными.

### 2.2 Второй регион

In [6]:
predicted_valid, target_valid, rmse = region(data_2)
print('Средний запас предсказанного сырья:', predicted_valid.mean())
print('Средний запас реального сырья:', target_valid.mean())
print('RMSE:', rmse)

Средний запас предсказанного сырья: 68.728546895446
Средний запас реального сырья: 68.72313602435997
RMSE: 0.8930992867756158


Тут ситуация другая, нежели с 1ым регионом, все предсказания по скважинам почти корректны.

### 2.3 Третий регион

In [7]:
predicted_valid, target_valid, rmse = region(data_3)
print('Средний запас предсказанного сырья:', predicted_valid.mean())
print('Средний запас реального сырья:', target_valid.mean())
print('RMSE:', rmse)

Средний запас предсказанного сырья: 94.96504596800489
Средний запас реального сырья: 94.88423280885438
RMSE: 40.02970873393434


Довольно сильный разброс по количеству сырья в скважине. Где-то модель прибавила лишние тысячи баррелей, где-то урезала.<br>
Но в совокупности получилось почти идентичное среднее действительному. 

<a id='3'></a>

# 3. Подготовка к расчёту прибыли

Сохраним в переменных все ключевые значения для расчетов.

In [8]:
budget = 10000000000
price_per_1000bar = 450000
kol_razrab_scvazhin = 200 # В условиях задачи написано, что именно 200 скважин разрабатываем в конечном итоге
kol_razved_scvazhin = 500 # Такое количество точек исследуют при разведке региона

# Рассчитаем достаточный объем для безубыточной разработки новой скважины
bezub_V = budget / price_per_1000bar / kol_razrab_scvazhin
print('Объем (в тыс. баррелей) в 1ой скважине для ее безубыточной разработки:', bezub_V)

Объем (в тыс. баррелей) в 1ой скважине для ее безубыточной разработки: 111.11111111111111


В итоге получаем, что во всех 200 новых скважинах должно быть не менее 111 единиц сырья для безубыточной разработки в регионе.

Со средними значениями объема скважин в регионах  - это не сравнимо, средние меньше. Но стоит понимать, что среднее - это среднее, т.е. возможно все же есть те скважины, которые в итоге суммарно принесут нам прибыль. Осталось их найти.

Поймем, какой из регионов принесет большую прибыль и принесет ли вообще.

<a id='4'></a>

# 4. Функция для расчета прибыли по выбранным скважинам и предсказаниям модели

In [10]:
# ОТБОР ЛУЧШЕ ПРОВОДИТЬ ПО ТАРГЕТУ.
def profit(subsample, target):

    # Выберем скважины с максимальными значениями предсказаний (разрабатывать будут kol_razrab_scvazhin = 200 лучших скважин)
    predicted_valid_selected = subsample.sort_values(ascending = False)#.head(kol_razrab_scvazhin)

    # Целевые значения объема сырья, соответствующие выбранным значениям предсказаний
    target_valid_selected = target[predicted_valid_selected.index][:kol_razrab_scvazhin]

    # Сумма объемов сырья выбранных скважин
    sum_scvazhin_max_V = target_valid_selected.sum()
    
    # Прибыль
    profit = price_per_1000bar * sum_scvazhin_max_V - budget
    
    # Переведем из рублей в млрд. руб
    profit = profit / 1000000000
    
    return profit

<a id='5'></a>

# 5. Расчёт прибыли и рисков 

### 5.1 Первый регион

In [11]:
from numpy.random import RandomState

state = RandomState(12345)

# Напишем функцию для подсчета прибыли и рисков для любого региона
def rachet(data):
    predicted_valid, target_valid, rmse = region(data)
    
    predicted_valid = pd.Series(predicted_valid)
    target_valid = pd.Series(target_valid).reset_index(drop = True)
    
    profits = []
    
    for i in range(1000):
        subsample = predicted_valid.sample(n = kol_razved_scvazhin, 
                            replace = True, 
                            random_state = state)
        target_valid_subsample = target_valid[subsample.index]
        profits.append(profit(subsample, target_valid_subsample))
        
    profits = pd.Series(profits)
    
    # Рассчитаем вероятность убытков
    count_minus = 0
    for i in profits.index:
        if profits[i] < 0:
            count_minus += 1
    risk = count_minus / len(profits)
        
    
    # Рассчитаем 95%-ый доверительный интервал
    lower = profits.quantile(0.025)
    upper = profits.quantile(0.975)
    
    print('С высокой вероятностью прибыль будет от {:0.3f} млрд.руб до {:0.3f} млрд.руб'.format(lower, upper))
    print('Риск убытков: {:0.1%}'.format(risk))
    print('Средняя прибыль: {:0.3f} млрд.руб'.format(profits.mean()))

rachet(data_1)

С высокой вероятностью прибыль будет от -0.102 млрд.руб до 0.948 млрд.руб
Риск убытков: 6.0%
Средняя прибыль: 0.426 млрд.руб


### 5.2 Второй регион

In [12]:
rachet(data_2)

С высокой вероятностью прибыль будет от 0.128 млрд.руб до 0.954 млрд.руб
Риск убытков: 0.3%
Средняя прибыль: 0.518 млрд.руб


### 5.3 Третий регион

In [13]:
rachet(data_3)

С высокой вероятностью прибыль будет от -0.116 млрд.руб до 0.990 млрд.руб
Риск убытков: 6.2%
Средняя прибыль: 0.420 млрд.руб


<a id='6'></a>

# 6. Общий вывод

Что ж, данные подготовили, анализ провели, пора делать выводы.

Исходя из получившихся значений, самый выгодный и менее рискованный для разработки регион - **второй**.

**Риск убытков: вероятность отрицательных прибылей у всех регионов меньше, чем 2.5%.**<br>
Однако, и тут выделился второй регион с самым высоким значением нижнего порога прибыли при 95%-ом доверительном интервале,
т.е. вероятность убытков у этого региона наименьшая. <br>

Также помним, что во втором регионе прослеживается сильная зависимость значения параметра 'f2' от объема сырья в скважине. Именно поэтому так мал разброс ошибок предсказаний объемов. 

**Копаем во 2-ом регионе.**